# Xenium Spatial Transcriptomics Analysis Workflow

This notebook provides a comprehensive workflow for analyzing **10X Genomics Xenium** spatial transcriptomics data using the SpatialData framework.

## Workflow Overview

1. **Project Setup** - Organize files and create directory structure
2. **Data Loading & Initial QC** - Import Xenium data and perform quality control
3. **Spatial Visualization** - Visualize cells in spatial context
4. **Quality Control Metrics** - Assess control probes and transcript quality
5. **Cell Filtering & Normalization** - Filter low-quality cells and normalize data
6. **Dimensionality Reduction & Clustering** - PCA, UMAP, and Leiden clustering
7. **Spatial Visualization** - Plot cells and clusters spatially
8. **Region of Interest (ROI) Analysis** - Zoom into specific tissue regions
9. **Optional: Resegmentation** - Improve cell segmentation using Cellpose

## Requirements

### Python Modules (must be in your working directory)
- `xenium.py` - Xenium data reader module
- `sbf.py` - Visualization helper functions
- `reseg.py` - Resegmentation pipeline (optional)
- `reseg_tools.py` - Resegmentation utilities (optional)
- `_constants.py` - Xenium data format constants

### Data Files
Xenium output directory contains:
- `transcripts.parquet` - Individual transcript coordinates
- `cells.parquet` - Cell metadata (centroids, areas)
- `cell_boundaries.parquet` - Cell boundary polygons
- `nucleus_boundaries.parquet` - Nucleus boundary polygons
- `cell_feature_matrix.h5` - Gene expression matrix
- `morphology_focus/` - High-resolution microscopy images (directory)
- `experiment.xenium` - Experiment metadata

---
## 1. Project Setup and File Organization

### Step 1.1: Create Project Directory Structure

Before starting analysis, we need to organize our data and code files properly.

**Required directory structure:**
```
project_folder/
├── xenium.py                # Xenium reader module
├── sbf.py                   # Visualization functions
├── reseg.py                 # Resegmentation module (optional)
├── reseg_tools.py           # Resegmentation tools (optional)
├── _constants.py            # Constants definitions
├── xenium_QC.py             # QC script (reference)
└── xenium_output/           # Xenium output directory
    ├── transcripts.parquet
    ├── cells.parquet
    ├── cell_boundaries.parquet
    ├── nucleus_boundaries.parquet
    ├── cell_feature_matrix.h5
    ├── experiment.xenium
    └── morphology_focus/    # Image directory
        ├── morphology_focus_0000.ome.tif
        ├── morphology_focus_0001.ome.tif
        ├── morphology_focus_0002.ome.tif
        └── morphology_focus_0003.ome.tif
```

### Step 1.2: File Organization Instructions

**For Xenium data from 10X Genomics:**

1. **Create main project folder** (e.g., `Xenium_Project/`)
2. **Copy Python modules** to the main folder:
   - `xenium.py`, `sbf.py`, `_constants.py`
   - Optional: `reseg.py`, `reseg_tools.py`
3. **Copy Xenium output directory** to the main folder
   - The entire Xenium output folder from the instrument/pipeline
   - Alternatively, you can point directly to the Xenium output location

**Important Notes:**
- All Xenium files must be in the same output directory
- Do not rename Xenium output files (they have specific names)
- The `morphology_focus/` directory contains multi-channel images:
  - Channel 0: DAPI (nuclear)
  - Channel 1: Boundary markers
  - Channel 2: RNA (18S)
  - Channel 3: Protein markers
- If you have post-Xenium images (H&E, IF), keep them in the same directory

### Step 1.3: Verify File Structure

In [ ]:
import os
import glob

# TODO: Update this path to your Xenium output directory
path = "/path/to/xenium/output/"

print("Checking Xenium output structure...\n")

# Check for required Python modules in parent directory
parent_dir = os.path.dirname(path) if path.endswith('/') else os.path.dirname(path + '/')
required_modules = ['xenium.py', 'sbf.py', '_constants.py']
optional_modules = ['reseg.py', 'reseg_tools.py', 'xenium_QC.py']

print("Required modules (in parent directory):")
for module in required_modules:
    if os.path.exists(os.path.join(parent_dir, module)):
        print(f"  ✓ {module}")
    else:
        print(f"  ✗ {module} - MISSING!")

print("\nOptional modules:")
for module in optional_modules:
    if os.path.exists(os.path.join(parent_dir, module)):
        print(f"  ✓ {module}")
    else:
        print(f"  - {module} - not found")

# Check for required Xenium files
print("\nXenium output files:")
required_files = {
    'transcripts.parquet': 'Transcript coordinates',
    'cells.parquet': 'Cell metadata',
    'cell_boundaries.parquet': 'Cell boundaries',
    'nucleus_boundaries.parquet': 'Nucleus boundaries',
    'cell_feature_matrix.h5': 'Gene expression matrix',
    'experiment.xenium': 'Experiment metadata'
}

all_present = True
for filename, description in required_files.items():
    filepath = os.path.join(path, filename)
    if os.path.exists(filepath):
        size = os.path.getsize(filepath) / (1024**2)  # Size in MB
        print(f"  ✓ {filename:<30} ({description}, {size:.1f} MB)")
    else:
        print(f"  ✗ {filename:<30} - MISSING!")
        all_present = False

# Check for morphology images
morphology_dir = os.path.join(path, 'morphology_focus')
if os.path.exists(morphology_dir):
    image_files = glob.glob(os.path.join(morphology_dir, '*.ome.tif'))
    print(f"\n✓ morphology_focus/ directory found with {len(image_files)} image(s)")
    
    # Check for expected channels
    expected_patterns = ['*_0000.ome.tif', '*_0001.ome.tif', '*_0002.ome.tif', '*_0003.ome.tif']
    for i, pattern in enumerate(expected_patterns):
        matching = glob.glob(os.path.join(morphology_dir, pattern))
        channel_names = ['DAPI (nuclear)', 'Boundary markers', '18S (RNA)', 'Protein markers']
        if matching:
            print(f"  ✓ Channel {i}: {channel_names[i]}")
        else:
            print(f"  - Channel {i}: {channel_names[i]} - not found")
else:
    print(f"\n✗ morphology_focus/ directory NOT FOUND!")
    all_present = False

# Check for optional post-Xenium images
print("\nOptional post-Xenium images:")
he_image = glob.glob(os.path.join(path, '*he_image.ome.tif'))
if_image = glob.glob(os.path.join(path, '*if_image.ome.tif'))

if he_image:
    print(f"  ✓ H&E image found: {os.path.basename(he_image[0])}")
else:
    print(f"  - H&E image not found")

if if_image:
    print(f"  ✓ IF image found: {os.path.basename(if_image[0])}")
else:
    print(f"  - IF image not found")

print("\n" + "="*60)
if all_present:
    print("✓ All required Xenium files are present!")
    print("You can proceed with the analysis.")
else:
    print("⚠ Some required files are missing!")
    print("Please ensure you have a complete Xenium output directory.")
print("="*60)

---
## 2. Setup and Imports

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import warnings
import logging

# Silence warnings for cleaner output
logging.basicConfig(level=logging.WARNING)
warnings.filterwarnings("ignore")

# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Image processing
import skimage.io as skio
import skimage.measure as skm
from skimage.transform import AffineTransform, warp

# Spatial transcriptomics libraries
import spatialdata as sd
from spatialdata_io import xenium
import spatialdata_plot
import scanpy as sc

# Custom modules (ensure these are in your working directory)
import xenium
from sbf import visualise_crop

print("✓ All imports successful!")

### Set Working Directory and Paths

**Important:** Update the `path` variable to point to your Xenium output directory.

In [ ]:
# TODO: Update this path to your Xenium output directory
path = "/path/to/xenium/output/"
os.chdir(path)

# Define paths
zarr_path = "Xenium_data.zarr"  # Output zarr file name
slide = "/flatFiles/SLIDENAME"  # Optional: path to slide-specific files

print(f"Working directory: {os.getcwd()}")
print("\nExpected Xenium files:")
print("  - transcripts.parquet")
print("  - cells.parquet")
print("  - cell_boundaries.parquet")
print("  - nucleus_boundaries.parquet")
print("  - cell_feature_matrix.h5")
print("  - morphology_focus/ (directory with images)")

---
## 2. Load Xenium Data into SpatialData Format

The `xenium.xenium()` function reads all Xenium output files and converts them into a unified **SpatialData** object.

**First Run:** This step may take 5-15 minutes depending on dataset size.

In [ ]:
# Ask if this is the first run
first_run = input("Is this the first run? (0: False, 1: True): ")

if first_run == '1':
    print("Loading Xenium data (this may take several minutes)...")
    sdata = xenium.xenium(path)
    
    # Save to zarr format for faster loading later
    sdata.write(zarr_path)
    print(f"✓ Data saved to {zarr_path}")
else:
    print("Skipping initial load (not first run)")

# Display data structure
print("\nSpatialData structure:")
print(sdata)

### Reload Data from Zarr (For Subsequent Sessions)

In [ ]:
# Load saved data
sdata = sd.read_zarr(zarr_path)
print("✓ Data loaded from zarr")

# Extract AnnData table (gene expression + metadata)
adata = sdata.tables["table"]

print(f"\nAnnData shape: {adata.shape}")
print(f"Cells: {adata.n_obs:,}")
print(f"Genes: {adata.n_vars}")
print(f"\nAvailable metadata columns:")
print(adata.obs.keys().tolist())

---
## 3. Basic Spatial Visualization

Xenium provides high-resolution spatial coordinates for each cell.

In [ ]:
# Extract spatial coordinates
xy = adata.obsm['spatial']

print(f"Coordinate range:")
print(f"  X: {xy[:, 0].min():.0f} to {xy[:, 0].max():.0f}")
print(f"  Y: {xy[:, 1].min():.0f} to {xy[:, 1].max():.0f}")

# Plot all cells
plt.figure(figsize=(12, 10))
plt.scatter(xy[:, 0], xy[:, 1], s=0.0001, alpha=0.5)
plt.title("All Cells - Spatial Distribution")
plt.xlabel("X coordinate (µm)")
plt.ylabel("Y coordinate (µm)")
plt.axis('equal')
plt.tight_layout()
plt.savefig('cells_spatial_distribution.png', format='png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Spatial distribution saved")

---
## 4. Quality Control Metrics

Xenium includes **control probes** and **codewords** to assess data quality:

- **Control probe counts** - Negative DNA probes (should be <5%)
- **Control codeword counts** - Negative decoding results (should be <5%)

High percentages indicate issues with probe specificity or decoding quality.

In [ ]:
# Calculate QC metrics
sc.pp.calculate_qc_metrics(adata, percent_top=(10, 20, 50, 150), inplace=True)

# Check for control probe columns
if 'control_probe_counts' in adata.obs.columns and 'control_codeword_counts' in adata.obs.columns:
    # Calculate percentages
    cprobes = (adata.obs["control_probe_counts"].sum() / adata.obs["total_counts"].sum() * 100)
    cwords = (adata.obs["control_codeword_counts"].sum() / adata.obs["total_counts"].sum() * 100)
    
    print(f"{'='*60}")
    print(f"Negative DNA probe count %: {cprobes:.2f}%")
    print(f"Negative decoding count %: {cwords:.2f}%")
    print(f"{'='*60}")
    
    # Quality warnings
    if cprobes > 5:
        print("⚠️  Warning: Control probe percentage is high (>5%)")
    else:
        print("✓ Control probe percentage is acceptable (<5%)")
        
    if cwords > 5:
        print("⚠️  Warning: Control codeword percentage is high (>5%)")
    else:
        print("✓ Control codeword percentage is acceptable (<5%)")
else:
    print("Note: Control probe/codeword columns not found in this dataset")
    print("This is normal for some Xenium output versions")

---
## 5. QC Plots - Cell-Level Statistics

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(20, 4))
fig.suptitle('Cell-Level Quality Metrics', fontsize=16)

# Total transcripts per cell
sns.histplot(adata.obs["total_counts"], kde=False, ax=axs[0], bins=50)
axs[0].set_title("Total Transcripts per Cell")
axs[0].set_xlabel("Total counts")
axs[0].axvline(100, color='r', linestyle='--', label='Filter threshold (100)')
axs[0].legend()

# Unique genes per cell
sns.histplot(adata.obs["n_genes_by_counts"], kde=False, ax=axs[1], bins=50)
axs[1].set_title("Unique Genes per Cell")
axs[1].set_xlabel("Number of genes")

# Cell area
if 'cell_area' in adata.obs.columns:
    sns.histplot(adata.obs["cell_area"], kde=False, ax=axs[2], bins=50)
    axs[2].set_title("Cell Area")
    axs[2].set_xlabel("Area (µm²)")
else:
    axs[2].text(0.5, 0.5, 'Cell area not available', ha='center', va='center')
    axs[2].set_title("Cell Area")

# Nucleus ratio
if 'nucleus_area' in adata.obs.columns and 'cell_area' in adata.obs.columns:
    nucleus_ratio = adata.obs["nucleus_area"] / adata.obs["cell_area"]
    sns.histplot(nucleus_ratio, kde=False, ax=axs[3], bins=50)
    axs[3].set_title("Nucleus Ratio")
    axs[3].set_xlabel("Nucleus / Cell area")
else:
    axs[3].text(0.5, 0.5, 'Nucleus area not available', ha='center', va='center')
    axs[3].set_title("Nucleus Ratio")

plt.tight_layout()
plt.savefig('cell_statistics.png', format='png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Cell statistics saved")

### Summary Statistics

In [ ]:
# Print summary statistics
print("Summary Statistics:")
print(f"{'='*60}")
print(f"Total cells: {adata.n_obs:,}")
print(f"Total genes: {adata.n_vars}")
print(f"\nTranscripts per cell:")
print(f"  Mean: {adata.obs['total_counts'].mean():.1f}")
print(f"  Median: {adata.obs['total_counts'].median():.1f}")
print(f"  Min: {adata.obs['total_counts'].min()}")
print(f"  Max: {adata.obs['total_counts'].max()}")
print(f"\nGenes per cell:")
print(f"  Mean: {adata.obs['n_genes_by_counts'].mean():.1f}")
print(f"  Median: {adata.obs['n_genes_by_counts'].median():.1f}")
print(f"  Min: {adata.obs['n_genes_by_counts'].min()}")
print(f"  Max: {adata.obs['n_genes_by_counts'].max()}")
print(f"{'='*60}")

---
## 6. Filtering and Normalization

**Filtering criteria:**
- Remove cells with <100 total transcripts
- Remove genes detected in <100 cells

**Note:** These thresholds should be adjusted based on your QC plots above.

In [ ]:
print(f"Original dimensions: {adata.shape}")
print(f"Cells: {adata.n_obs:,}, Genes: {adata.n_vars}")

# Filter cells
sc.pp.filter_cells(adata, min_counts=100)
print(f"\nAfter filtering cells (min_counts=100): {adata.shape}")
print(f"Cells: {adata.n_obs:,}, Genes: {adata.n_vars}")
print(f"Removed: {adata.shape[0]} cells")

# Filter genes
sc.pp.filter_genes(adata, min_cells=100)
print(f"\nAfter filtering genes (min_cells=100): {adata.shape}")
print(f"Cells: {adata.n_obs:,}, Genes: {adata.n_vars}")

# Store raw counts
adata.layers["counts"] = adata.X.copy()
print("\n✓ Raw counts stored in layer 'counts'")

# Normalize and log-transform
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)

print("✓ Normalization and log-transformation complete!")

---
## 7. Dimensionality Reduction and Clustering

Standard single-cell analysis workflow:
1. **PCA** - Principal component analysis
2. **Neighbors** - K-nearest neighbors graph
3. **UMAP** - Uniform Manifold Approximation and Projection
4. **Leiden** - Community detection clustering

In [ ]:
print("Running dimensionality reduction...")
print("This may take a few minutes for large datasets.")

# PCA
sc.pp.pca(adata)
print("✓ PCA complete")

# Neighbors
sc.pp.neighbors(adata)
print("✓ Neighbor graph computed")

# UMAP
sc.tl.umap(adata)
print("✓ UMAP complete")

# Leiden clustering
sc.tl.leiden(adata)
n_clusters = len(adata.obs['leiden'].unique())
print(f"✓ Leiden clustering complete - {n_clusters} clusters identified")

### UMAP Visualization

In [ ]:
# UMAP colored by QC metrics and clusters
sc.pl.umap(
    adata, 
    color=["total_counts", "n_genes_by_counts", "leiden"], 
    wspace=0.4,
    save='_xenium_qc.png'
)

print("✓ UMAP plots saved to 'figures/umap_xenium_qc.png'")

---
## 8. Spatial Visualization of Clusters

In [ ]:
# Add global coordinates to metadata
adata.obs["x_global_px"] = adata.obsm['spatial'][:, 0]
adata.obs["y_global_px"] = adata.obsm['spatial'][:, 1]

# Plot clusters in spatial context
plt.figure(figsize=(14, 12))

g = sns.scatterplot(
    x="x_global_px", 
    y="y_global_px", 
    s=2, 
    marker='.',
    data=adata.obs, 
    hue='leiden', 
    palette="tab20",
    alpha=0.7
)

# Adjust legend
handles, labels = g.get_legend_handles_labels()
for h in handles:
    h.set_markersize(h.get_markersize() * 8)

plt.legend(
    handles, 
    labels, 
    bbox_to_anchor=(1.05, 1), 
    loc='upper left',
    borderaxespad=0,
    ncol=2,
    title='Leiden Cluster'
)

plt.title("Spatial Distribution of Clusters", fontsize=16, pad=20)
plt.xlabel("X coordinate (µm)")
plt.ylabel("Y coordinate (µm)")
plt.axis('equal')
plt.tight_layout()
plt.savefig('spatial_clusters.png', format='png', dpi=600, bbox_inches='tight')
plt.show()

print("✓ Spatial cluster plot saved to 'spatial_clusters.png'")

---
## 9. Region of Interest (ROI) Analysis

For detailed inspection of specific tissue regions, we can "crop" to a smaller area.

**Instructions:**
1. Look at the spatial plot above to identify interesting regions
2. Note the X and Y coordinates
3. Define a bounding box using min/max coordinates

In [ ]:
# TODO: Update these coordinates based on your region of interest
size = 1500  # Size of the ROI in pixels
min_co = [24500, 9000]  # Bottom-left corner [x, y]
max_co = [min_co[0] + size, min_co[1] + size]  # Top-right corner

print(f"ROI coordinates:")
print(f"  X range: {min_co[0]} to {max_co[0]}")
print(f"  Y range: {min_co[1]} to {max_co[1]}")
print(f"  Size: {size} x {size} pixels")

# Visualize the ROI
visualise_crop(sdata, min_co, max_co)

print("\n✓ ROI visualization complete!")
print("The plot shows:")
print("  1. morphology_focus - High-resolution microscopy image")
print("  2. cell_labels - Segmented cell masks")
print("  3. transcripts - Individual transcript locations")
print("  4. cell_boundaries - Cell boundary polygons")

### Interactive ROI Selection

In [ ]:
# Filter cells within ROI
roi_cells = adata.obs[
    (adata.obs['x_global_px'] >= min_co[0]) & 
    (adata.obs['x_global_px'] <= max_co[0]) &
    (adata.obs['y_global_px'] >= min_co[1]) & 
    (adata.obs['y_global_px'] <= max_co[1])
]

print(f"Cells in ROI: {len(roi_cells):,}")
print(f"\nCluster distribution in ROI:")
print(roi_cells['leiden'].value_counts().sort_index())

---
## 10. Cluster Characterization

Find marker genes for each cluster to help with cell type annotation.

In [ ]:
# Find marker genes using Wilcoxon rank-sum test
print("Finding marker genes for each cluster...")
print("This may take several minutes for large datasets.")

sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
print("✓ Marker gene analysis complete")

# Display top 5 markers per cluster
print("\nTop 5 marker genes per cluster:")
sc.pl.rank_genes_groups(adata, n_genes=5, sharey=False, save='_marker_genes.png')

### Export Marker Genes

In [ ]:
# Extract marker gene results
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names

# Create a dataframe with top 20 markers per cluster
marker_dict = {}
for group in groups:
    marker_dict[f'Cluster_{group}_genes'] = result['names'][group][:20]
    marker_dict[f'Cluster_{group}_scores'] = result['scores'][group][:20]
    marker_dict[f'Cluster_{group}_pvals'] = result['pvals_adj'][group][:20]

marker_df = pd.DataFrame(marker_dict)
marker_df.to_csv('cluster_marker_genes.csv', index=False)
print("✓ Marker genes saved to 'cluster_marker_genes.csv'")

---
## 11. Visualize Specific Genes

Plot expression of genes of interest in spatial context.

In [ ]:
# TODO: Update with genes of interest from your panel
genes_of_interest = ['CD3E', 'CD68', 'KRT8', 'VIM']  # Example: T-cells, macrophages, epithelial, mesenchymal

# Check which genes are available
available_genes = [g for g in genes_of_interest if g in adata.var_names]
if len(available_genes) == 0:
    print("⚠️  None of the specified genes found in dataset")
    print(f"\nFirst 20 genes in dataset:")
    print(adata.var_names[:20].tolist())
else:
    print(f"Found {len(available_genes)} genes: {available_genes}")
    
    # UMAP visualization
    sc.pl.umap(adata, color=available_genes, cmap='viridis', save='_gene_expression_umap.png')
    
    # Spatial visualization
    for gene in available_genes:
        plt.figure(figsize=(12, 10))
        
        # Get expression values
        gene_idx = adata.var_names.tolist().index(gene)
        expr = adata[:, gene].X.toarray().flatten() if hasattr(adata.X, 'toarray') else adata[:, gene].X.flatten()
        
        # Plot
        scatter = plt.scatter(
            adata.obs['x_global_px'],
            adata.obs['y_global_px'],
            c=expr,
            s=0.5,
            cmap='viridis',
            alpha=0.7
        )
        
        plt.colorbar(scatter, label='Expression (log)')
        plt.title(f"{gene} Expression", fontsize=16, pad=20)
        plt.xlabel("X coordinate (µm)")
        plt.ylabel("Y coordinate (µm)")
        plt.axis('equal')
        plt.tight_layout()
        plt.savefig(f'spatial_{gene}_expression.png', format='png', dpi=300, bbox_inches='tight')
        plt.show()
    
    print(f"\n✓ Gene expression plots saved")

---
## 12. Save Processed Data

In [ ]:
# Save processed AnnData object
adata.write_h5ad("processed_xenium_data.h5ad")
print("✓ Processed data saved to 'processed_xenium_data.h5ad'")

# Save cluster assignments and coordinates
cluster_df = adata.obs[['leiden', 'x_global_px', 'y_global_px']].copy()
if 'cell_area' in adata.obs.columns:
    cluster_df['cell_area'] = adata.obs['cell_area']
if 'nucleus_area' in adata.obs.columns:
    cluster_df['nucleus_area'] = adata.obs['nucleus_area']

cluster_df.to_csv("cell_clusters.csv")
print("✓ Cluster assignments saved to 'cell_clusters.csv'")

# Save cluster statistics
cluster_stats = adata.obs.groupby('leiden').agg({
    'total_counts': ['mean', 'std'],
    'n_genes_by_counts': ['mean', 'std']
}).round(2)
cluster_stats['n_cells'] = adata.obs.groupby('leiden').size()
cluster_stats.to_csv('cluster_statistics.csv')
print("✓ Cluster statistics saved to 'cluster_statistics.csv'")

---
## 13. Optional: Resegmentation with Cellpose

If the default segmentation quality is poor, you can improve it using Cellpose.

**Requirements:**
- `reseg.py` module
- `reseg_tools.py` module
- Cellpose installed (`pip install cellpose`)

**Warning:** This is computationally intensive and may take hours for large datasets!

In [ ]:
# Uncomment to run resegmentation

# from reseg_tools import qc_metric, qc_plot, filter_norm, vis_segmentation
# from reseg import Resegmentation

# # Parameters for resegmentation
# channel_names = ['DAPI', 'Boundary', 'RNA', 'Protein']  # Adjust based on your data
# channels_to_use = ['DAPI']  # Use nuclear channel for segmentation
# factor_rescale = 8  # Downscale factor for faster processing

# # Initialize resegmentation pipeline
# pipe = Resegmentation(
#     path=path,
#     zarr_name=zarr_path,
#     output='resegmentation_ready.png',
#     factor_rescale=factor_rescale,
#     image_name='morphology_focus',
#     label_name='cell_labels',
#     shape_name='cell_boundaries',
#     point_name='transcripts'
# )

# # Preprocess images
# pipe.preprocess_image(channel_names, channels_to_use)

# # Run Cellpose segmentation
# pipe.run_cellpose(
#     flow_threshold=1.2,
#     cellprob_threshold=-3,
#     tile_overlap=0.15
# )

# # Update SpatialData object with new segmentation
# pipe.update_spatialdata()

# # Load resegmented data
# sdata_reseg = sd.read_zarr('updated_' + zarr_path)
# adata_reseg = sdata_reseg.tables['table']

# # Run QC on resegmented data
# qc_metric(adata_reseg)
# qc_plot(adata_reseg, sdata_reseg)
# filter_norm(adata_reseg)

# print("✓ Resegmentation complete!")

---
## Summary

This notebook covered:

1. ✓ Loading Xenium data from output files
2. ✓ Basic spatial visualization
3. ✓ Quality control metrics (control probes, codewords)
4. ✓ Cell and gene filtering
5. ✓ Normalization and log-transformation
6. ✓ Dimensionality reduction (PCA, UMAP)
7. ✓ Clustering (Leiden)
8. ✓ Spatial visualization of clusters
9. ✓ Region of interest analysis
10. ✓ Marker gene identification
11. ✓ Gene expression visualization
12. ✓ Optional resegmentation

## Next Steps

- **Cell type annotation:** Annotate clusters based on marker genes
- **Spatial statistics:** Neighborhood enrichment, co-localization analysis
- **Differential expression:** Compare gene expression between regions/conditions
- **Integration:** Combine with other modalities (H&E, immunofluorescence)
- **Trajectory analysis:** Identify developmental or differentiation paths

## References

- [SpatialData Documentation](https://spatialdata.scverse.org/)
- [Scanpy Documentation](https://scanpy.readthedocs.io/)
- [Xenium Platform](https://www.10xgenomics.com/platforms/xenium)
- [Xenium Analysis Guide](https://www.10xgenomics.com/support/software/xenium-onboard-analysis/latest)